In [1]:
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score
import RetinaVVS.RetinaVVS_class as RetinaVVS_class
from torchvision.datasets import CIFAR10
from torchvision import transforms
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import numpy as np
import torch
import json

In [2]:
def get_hparams(path):
    with open(path) as f:
        r = f.read()
    return json.loads(r)

In [ ]:
# Get hyperparameters
# TODO: add hyperparameter automatic saving with torch lightning
path = "Best_Models/RetinaVVS/RetChans32_VVSLayers4/parameters.txt"
weight_path = "Best_Models/RetinaVVS/RetChans32_VVSLayers4/weights.tar"
hparams = get_hparams(path)

# Get model
model = RetinaVVS_class.RetinaVVS(hparams)
model.load_state_dict(torch.load(weight_path))
model.eval()
print ("Model grabbed")

In [ ]:
# Get test dataloader
transform = transforms.Compose([
        transforms.Grayscale(1),
        transforms.ToTensor()
    ])
test_data = CIFAR10(root="data/CIFAR10", download=False, train=False, transform=transform)
test_loader = DataLoader(test_data, batch_size=32, num_workers=12)
print ("Dataloader acquired")

In [ ]:
# Prediction
y_true = torch.cat([labels for _, labels in test_loader]).reshape(-1).numpy()
y_pred = torch.cat([model(images).argmax(axis=-1) for images, _ in test_loader]).reshape(-1).numpy()
print ("Predictions calculated")

In [ ]:
model_acc = accuracy_score(y_true, y_pred)
print("Model accuracy:", model_acc